In [1]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from utils.database import CreateDatabase, SearchDatabase
from utils.model_rag import ModelRagRanker
from transformers import pipeline
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

In [2]:
def process_dataset(sample):
    sample['content'] = sample['content'].replace('\xa0', '')
    return sample

In [3]:
dataset = load_dataset('blinoff/kinopoisk')['train']
dataset = dataset.map(process_dataset)

In [4]:
dataset[0]

{'part': 'top250',
 'movie_name': 'Блеф (1976)',
 'review_id': '17144',
 'author': 'Come Back',
 'date': datetime.datetime(2011, 9, 24, 0, 0),
 'title': 'Плакали наши денежки ©',
 'grade3': 'Good',
 'grade10': '10',
 'content': '\n"Блеф»—одна измоих самых любимых комедий.\n\nЭтот фильм янаверно смотрел разсто, нетя конечно блефую, явидел егокуда больше. Немогу невыразить своё восхищение главными действующими лицами этого фильма. Начну сАдриано Челентано длякоторого какя считаю этолучшая роль вкино. Великолепный актёр, неплохой певец, странно чтона егородине вИталии егопесни мало ктослушает. Нуя думаю чтои итальянцы ифранцузы привыкли ктому, чтоу насдо сихих актёры популярней чемдаже насвоей родине. Да, такой вотпарадокс. Челентано конечно профессионал своего дела, комик ссерьёзным выражением лица. Онсмешон ещёи потому, чтоодновременно таксеръёзен. Адриано браво!\n\nАтеперь несколько слов обЭнтони Куине. Датот самый горбун изНотр-дама. Собор Парижской Богоматери, оригинальная версия, кт

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
generation_pipeline = pipeline(
    'text-generation',
    model='Qwen/Qwen2-1.5B-Instruct',
    device=device,
    torch_dtype=torch.float16
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda


In [7]:
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large", model_kwargs={'torch_dtype': torch.float16})

In [8]:
cross_encoder = HuggingFaceCrossEncoder(
    model_name='amberoad/bert-multilingual-passage-reranking-msmarco',
    model_kwargs={'device': device}
)

In [ ]:
create_db = CreateDatabase(
    embedding_model=embedding_model,
    dataset=dataset,
    device=device,
    collection_name='kinopoisk_e5',
    path='database'    
)

create_db.create()

 92%|█████████▏| 33814/36591 [19:25<01:43, 26.85it/s]

In [ ]:
model = ModelRagRanker(
    generation_pipeline=generation_pipeline,
    embedding_model=embedding_model,
    cross_encoder=cross_encoder,
    path_client='database',
    device=device,
    collection_name='kinopoisk_e5',
    temperature = 0.8,
    top_p = 0.6,
)

In [ ]:
query = 'В каких 3 фильмах играл Роберт де Ниро?'

In [ ]:
answer, context = model.predict(query)
print(answer)